## Import training/test files in Pandas

In [ ]:
import pandas as pd
import os 

print(os.getcwd())

In [ ]:
train_data = pd.read_csv(os.path.join(os.getcwd() + '/data/training_csv'))
test_data = pd.read_csv(os.path.join(os.getcwd() + '/data/test_csv'))

In [ ]:
# Note: we are enabling eager execution for debugging!

import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

## example code

In [ ]:
# Example code for handling datasets
import matplotlib.pyplot as plt

# Load filenames and labels
filenames = tf.constant(train_data.iloc[:, 0].tolist())
labels = tf.constant(train_data.iloc[:, 1:].values)

# Add to a dataset object
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# We can debug using eager execution
for img, labels in dataset.batch(4).take(1):
    print(img)
    print(labels)

## parse function from csv

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.

def _parse_function(filename, label):
    print(filename)
    image_string = tf.read_file(filename) 
    print(image_string)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3) 
    image_resized = tf.image.resize_images(image_decoded, [160, 160])
    image_shape = tf.cast(tf.shape(image_decoded), tf.float32)
    label = tf.concat([label[:]], axis=0)
    return {"x": image_resized}, label

## Make Dataset

In [ ]:
# This snippet is adapted from here: https://www.tensorflow.org/guide/datasets
def input_fn(dataframe, is_eval=False):

    # Load the list of files
    filenames = tf.constant(dataframe.iloc[:, 0].tolist())

    # Load the labels
    labels = tf.constant(dataframe.iloc[:, 1:].values)

    # Build the dataset with image processing on top of it
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(_parse_function)

    # Add shuffling and repeatition if training
    if is_eval:
        dataset = dataset.batch(64)
    else:
        dataset = dataset.repeat().shuffle(1000).batch(64)

    return dataset

In [ ]:
# Check the image & some labels
import matplotlib.pyplot as plt
for (imgs, labels) in input_fn(train_data, is_eval=True).take(1):
    plt.imshow(imgs['x'][0] / 255)
    print(labels[0])
    print(labels[1])

## extract feature

In [ ]:
def extract_features(features):
    # Input layer
    input_layer = tf.reshape(features["x"], [-1, 160, 160, 3])

    # First convolutive layer
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=16, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Second convolutive layer
    conv2 = tf.layers.conv2d(inputs=pool1, filters=48, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Third convolutive layer
    conv3 = tf.layers.conv2d(inputs=pool2, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    # Fourth convolutive layer
    conv4 = tf.layers.conv2d(inputs=pool3, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    # Fifth convolutive layer
    conv5 = tf.layers.conv2d(inputs=pool4, filters=128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)

    # Sixth convolutive layer
    conv4 = tf.layers.conv2d(inputs=pool3, filters=128, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)

    # Dense Layer (Fully Connected Layer)
    flat = tf.reshape(conv4, [-1, 20 * 20 * 128])
    dense = tf.layers.dense(inputs=flat, units=100, activation=tf.nn.relu)
  
    return dense

## input_fn & main estimator model

In [ ]:
# Need to disable the eager execution at this point
import numpy as np
import tensorflow as tf

def multi_input_fn(data, is_eval=False):
    features, labels = input_fn(data, is_eval=is_eval).make_one_shot_iterator().get_next()
    return features, {'shape': labels[:, 0], 'cover': labels[:, 1], 'charm': labels[:, 2], 'pattern': labels[:, 3]}

In [ ]:
def multi_head_cnn_model_fn(features, labels, mode):

    dense = extract_features(features)

    # Predictions for each task
    logits_shape = tf.layers.dense(inputs=dense, units=11)
    logits_cover = tf.layers.dense(inputs=dense, units=3)
    logits_charm = tf.layers.dense(inputs=dense, units=6)
    logits_pattern = tf.layers.dense(inputs=dense, units=12)
    
    # Make predictions
    
    predicted_shape_class = tf.argmax(logits_shape, 1)
    predicted_cover_class = tf.argmax(logits_cover, 1)
    predicted_charm_class = tf.argmax(logits_charm, 1)
    predicted_pattern_class = tf.argmax(logits_pattern, 1)
    
    logits = {'shape': logits_shape, 'cover': logits_cover, 
              'charm': logits_charm, 'pattern': logits_pattern}
    
    outputs = {
        "predicted_shape_class": predicted_shape_class,
        "predicted_cover_class": predicted_cover_class,
        "predicted_charm_class": predicted_charm_class,
        "predicted_pattern_class": predicted_pattern_class,
        "logits": logits
    }
    
    # Optimizer
    optimizer = tf.train.AdamOptimizer()
    
    # We just want the predictions
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=outputs)

    # If not in mode.PREDICT, compute the loss 
    shape_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels['shape'], logits=logits_shape)
    cover_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels['cover'], logits=logits_cover)
    charm_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels['charm'], logits=logits_charm)
    pattern_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels['pattern'], logits=logits_pattern)
    total_loss = shape_loss + cover_loss + charm_loss + pattern_loss

    # TRAIN MODE 
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=total_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=total_loss, train_op=train_op)

    # If not PREDICT or TRAIN, then we are evaluating the model
    eval_metric_ops = {
        "shape_accuracy": tf.metrics.accuracy(
            labels=labels['shape'], predictions=outputs["predicted_shape_class"]), 
        "cover_accuracy": tf.metrics.accuracy(
            labels=labels['cover'], predictions=outputs["predicted_cover_class"]), 
        "charm_accuracy": tf.metrics.accuracy(
            labels=labels['charm'], predictions=outputs["predicted_charm_class"]), 
        "pattern_accuracy": tf.metrics.accuracy(
            labels=labels['pattern'], predictions=outputs["predicted_pattern_class"]), 
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=total_loss, eval_metric_ops=eval_metric_ops)
    

## estimator 선언 및 training / evaluation / test

In [ ]:
multitask_classifier = tf.estimator.Estimator(model_fn=multi_head_cnn_model_fn)

In [ ]:
multitask_classifier.train(input_fn=lambda: multihead_input_fn(train_data), steps=10000)

In [ ]:
multitask_classifier.evaluate(input_fn=lambda: multihead_input_fn(test_data, is_eval=True))

In [ ]:
p = list(multitask_classifier.predict(lambda: input_fn(test_data, is_eval=True)))
print(p[0])

## restore trained model 

In [ ]:
with tf.Session() as sess :

    # Saver instance 를 생성한다.
    # Saver.restore(sess, ckpt_path)

    saver = tf.train.import_meta_graph('/tmp/tmpv28t5kpm/model.ckpt-5563.meta')
    saver.restore(sess, tf.train.latest_checkpoint('/tmp/tmpv28t5kpm/'))